In [1]:
# -- IMPORTS START --
import pandas as pd
import glob
import re
import os
import sys
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split

import datetime
# -- IMPORTS END --

# enable zooming into graphs
%matplotlib notebook
plt.rcParams['figure.figsize'] = [9, 6] # width, height in inches

### Helper Functions:

**unix_to_date:**
- Converts Epoch(Unix) Time into traditional DateTime information

**starter:**
- applies all necessary cleaning to eliminate null instances, unnecessary columns, etc.
- returns working DataFrame with relevant information

In [16]:
def unix_to_date(x):
    f = float(x)
    full = datetime.datetime.fromtimestamp(int(str(f)[:-5]))
    full = full.replace(microsecond=int(float(str(f)[-5:]))*1000)
    return full


In [17]:
def starter_acg(filepath):
    df = pd.read_csv(filepath)
    df.dropna(inplace=True)
    df['Time'] = df['t_unix'].apply(unix_to_date)
    df = df.drop(['t_unix','t_Android','a'], axis=1)
    df = df.reindex(columns=['Time', 'x', 'y', 'z'])

    return df

In [23]:
b_acg = starter_acg("data/Backhand/WEAR_ACG.csv")

f_acg = starter_acg("data/Forehand/WEAR_ACG.csv")

s_acg = starter_acg("data/Serve/WEAR_ACG.csv")

s_acg.head()



,Time,x,y,z
0,2023-11-27 16:52:03.540,-3.250764,6.453616,3.097448
1,2023-11-27 16:52:03.540,-3.250764,6.453616,3.097448
2,2023-11-27 16:52:03.625,-3.250764,6.453616,3.097448
3,2023-11-27 16:52:03.625,-4.371882,6.685985,5.710995
4,2023-11-27 16:52:03.625,-4.338344,6.685985,5.710995
